In [13]:
import psycopg2

conn = psycopg2.connect(database="postgres", user='postgres', password='Admin', host="localhost", port=5432)

cur = conn.cursor()

sql = ''' create table student_details1(student_id int, student_name varchar(30),
#         cgpa decimal)'''

# cur.execute(sql)
print("Table Successfully Created..")

data_list = [(11,'Krut',8),(12,'Vedansh',8.1),(13,'Harsh',7.5)]

# cur.executemany('INSERT INTO student_details1 VALUES(%s,%s,%s)',data_list)

s1 = '''select * from student_details1'''
cur.execute(s1)
for i in cur.fetchall():
    print(i)


s2 = '''UPDATE student_details1 SET cgpa = cgpa + (cgpa * 0.005)''' 
cur.execute(s2)

s3 = '''DELETE FROM student_details1 WHERE cgpa <= %s'''
dele = (18,)

cur.execute(s3,dele)
conn.commit()
conn.close()

Table Successfully Created..
(11, 'Krut', Decimal('18.18045000'))
(12, 'Vedansh', Decimal('18.407705625'))


In [8]:
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker

engine = db.create_engine('postgresql+psycopg2://postgres:Admin\
@localhost/postgres')

conn = engine.connect()
metadata = db.MetaData()
emp = db.Table('employee',metadata, db.Column('id', db.Integer, primary_key=True),  
    db.Column('name', db.String(50)),                    
    db.Column('salary', db.Float(),default=100.0))

metadata.create_all(engine)

# query = db.insert(emp).values(id = 1,name = 'Krut', salary = 20500)
# curr = conn.execute(query)
# query = db.insert(emp) 
# values_list = [{'id':'2', 'name':'ram', 'salary':80000},
#                {'id':'3', 'name':'ramesh', 'salary':70000}]
# conn.execute(query,values_list)
print("Done Added Employees...")

# query1 = db.update(emp).values(salary = 110000).where(emp.columns.name == 'Krut')
# conn.execute(query1)

query1 = db.delete(emp).where(emp.columns.salary <= 10000)
conn.execute(query1)
# query = conn.execute(db.select(emp)).fetchall()
# print(query)
conn.commit()
conn.close()

Done Added Employees...


In [15]:
# Task 1: Advanced Querying with SQLAlchemy
# Description: Create a products table with columns for id, name, category, and price. Insert some
# sample data and write queries to:
# Retrieve all products in a specific category.
# Retrieve products with a price above a certain threshold.
# Retrieve the average price of products in each category.


import sqlalchemy as db
from sqlalchemy.orm import sessionmaker

engine = db.create_engine('postgresql+psycopg2://postgres:Admin\
@localhost/postgres')

conn = engine.connect()
metadata = db.MetaData()
emp = db.Table('Products',metadata, db.Column('id', db.Integer, primary_key=True),  
    db.Column('name', db.String(50)),                    
    db.Column('category', db.String(50)),
    db.Column('price',db.Float(),default=0.0))
metadata.create_all(engine)

products_data = [{'name': 'Laptop', 'category': 'Electronics', 'price': 1000}, {'name': 'Smartphone',
'category': 'Electronics', 'price': 700}, {'name': 'Table', 'category': 'Furniture', 'price': 150}, {'name':
'Chair', 'category': 'Furniture', 'price': 85}]

query = db.insert(emp) 
conn.execute(query,products_data)


# Query 1: Retrieve all products in 'Electronics' category.
# Expected Output: [('Laptop', 'Electronics', 1000), ('Smartphone', 'Electronics', 700)]

query1 = db.select(emp).where(emp.columns.category == 'Electronics')
result = conn.execute(query1)
result = result.fetchall()
print(result)


# Query 2: Retrieve products with price > 500.
# Expected Output: [('Laptop', 'Electronics', 1000), ('Smartphone', 'Electronics', 700)]

query1 = db.select(emp).where(emp.columns.price > 500)
result = conn.execute(query1)
result = result.fetchall()
print(result)


# Query 3: Retrieve average price of products in each category.
# Expected Output: [('Electronics', 850.0), ('Furniture', 117.5)]

query1 = db.select([emp.columns.category,db.func.avg(emp.columns.price)]).group_by(emp.columns.category)
result = conn.execute(query1)
result = result.fetchall()
print(result)

conn.commit()
conn.close()



[(1, 'Laptop', 'Electronics', 1000.0), (2, 'Smartphone', 'Electronics', 700.0)]
[(1, 'Laptop', 'Electronics', 1000.0), (2, 'Smartphone', 'Electronics', 700.0)]


In [69]:
# Task 1: Advanced Querying with SQLAlchemy

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, func
from sqlalchemy.ext.declarative import declarative_base

db_url = 'postgresql+psycopg2://postgres:Admin\
@localhost/postgres'

engine = db.create_engine(db_url)

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()
class User(Base):
    __tablename__ = 'Task1'
    id = Column(Integer,primary_key=True,autoincrement=True)
    name = Column(String,nullable=False)
    category = Column(String,nullable=False)
    price = Column(Integer,default=0)


Base.metadata.create_all(engine)
products_data = [
   User(name = 'Laptop', category = 'Electronics', price = 1000), 
   User(name = 'Smartphone', category = 'Electronics', price = 700), 
   User(name = 'Table', category = 'Furniture', price = 150),
   User(name = 'Chair', category = 'Furniture', price = 85)]

# Insert sample data:
# session.add_all(products_data)
print("added....")

# Query 1: Retrieve all products in 'Electronics' category.
users = session.query(User).filter_by(category="Electronics").all()
for user in users:
    print(f"Electronics category products are: {user.id}, {user.name}, {user.category}, {user.price}")

# Query 2: Retrieve products with price > 500.
users = session.query(User).filter(User.price > 500).all()
for user in users:
    print(f"Products whose price are more than Rs.500 are: {user.id}, {user.name}, {user.category}, {user.price}")

# Query 3: Retrieve average price of products in each category
avg_prices = session.query(User.category,func.avg(User.price).label('average_price')).group_by(User.category).all()
for category, avg_price in avg_prices:
    print(f"The average price of products in each {category} is: {avg_price}")

session.commit()


added....
Electronics category products are: 1, Laptop, Electronics, 1000
Electronics category products are: 2, Smartphone, Electronics, 700
Products whose price are more than Rs.500 are: 1, Laptop, Electronics, 1000
Products whose price are more than Rs.500 are: 2, Smartphone, Electronics, 700
The average price of products in each Furniture is: 117.5000000000000000
The average price of products in each Electronics is: 850.0000000000000000


C:\Users\Dell\AppData\Local\Temp\ipykernel_13304\3740396383.py:16: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [50]:
# Task 2: Implementing Caching with SQLAlchemy Queries

import time
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
import functools as ft

db_url = 'postgresql+psycopg2://postgres:Admin\
@localhost/postgres'

engine = db.create_engine(db_url)

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()


class User(Base):
    __tablename__ = 'Products3'
    id = Column(Integer,primary_key=True,autoincrement=True)
    name = Column(String,nullable=False)
    category = Column(String,nullable=False)
    price = Column(Integer,default=0)


@ft.lru_cache(maxsize=128)
def querys(pid):
    user = session.query(User).get(pid)
    if user:
        return {
            'id': user.id,
            'name': user.name,
            'category': user.category,
            'price': user.price
        }
    return "No Data Found"

Base.metadata.create_all(engine)
products_data = [
   User(name = 'Laptop', category = 'Electronics', price = 1000), 
   User(name = 'Smartphone', category = 'Electronics', price = 700), 
   User(name = 'Table', category = 'Furniture', price = 150),
   User(name = 'Chair', category = 'Furniture', price = 85)]

# Insert sample data:
session.add_all(products_data)
print("added....")


# Measure time for the first call
# Output: (1, 'Laptop', 'Electronics', 1000)
# Time taken for second call: ~0 seconds

start_time = time.time()
user = querys(4)
end_time = time.time()
print(f"Time taken for first call: {end_time - start_time} seconds")

# Measure time for the second call (should be fast due to caching):
# Output: (1, 'Laptop', 'Electronics', 1000)
# Time taken for second call: ~0 seconds

start_time = time.time()
user = querys(4)
end_time = time.time()
print(f"Time taken for first call: {end_time - start_time} seconds")

    
# Check cache statistics:
# CacheInfo: CacheInfo(hits=1, misses=1, maxsize=128, currsize=1)

cache_info = querys.cache_info()
print(f"CacheInfo: {cache_info}")

session.commit()
print("Table created and user added!")


added....
Time taken for first call: 0.0054891109466552734 seconds
Time taken for first call: 0.0 seconds
CacheInfo: CacheInfo(hits=1, misses=1, maxsize=128, currsize=1)
Table created and user added!


C:\Users\Dell\AppData\Local\Temp\ipykernel_13304\1843746678.py:18: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
C:\Users\Dell\AppData\Local\Temp\ipykernel_13304\1843746678.py:31: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  user = session.query(User).get(pid)


In [91]:
# Task 3: Handling Relationships in SQLAlchemy

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

db_url = 'postgresql+psycopg2://postgres:Admin\
@localhost/postgres'

engine = db.create_engine(db_url)

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

class User(Base):
    __tablename__ = 'books1'
    id = Column(Integer,primary_key=True,autoincrement=True)
    title = Column(String,nullable=False)
    author_id = Column(Integer, ForeignKey("authors1.id"))
    author = relationship("Author", back_populates="book")

class Author(Base):
    __tablename__ = 'authors1'
    id = Column(Integer,primary_key=True,autoincrement=True)
    name = Column(String,nullable=False)
    book = relationship("User", back_populates="author")


Base.metadata.create_all(engine)

authors_data = [Author(name = 'J.K. Rowling',),
                Author(name = 'George R.R. Martin',)]

books_data = [User(title = 'Harry Potter and the Philosopher\'s Stone',author_id = 1,),
              User(title = 'Harry Potter and the Chamber of Secrets',author_id = 1),
              User(title = 'A Game of Thrones',author_id = 2),
              User(title= 'A Clash of Kings',author_id = 2)]

# session.add_all(authors_data)
# session.add(books_data)

print("added....")

books_by_author = session.query(User).join(Author).filter(Author.name == 'J.K. Rowling').all()
print("The Books Written by J.K. Rowling are:")
for i in books_by_author:
    print(f"{i.title}")


author_details = session.query(User).join(Author).filter(Author.name == 'George R.R. Martin').all()
print("The Author Details are:-")
for i in author_details:
    print(f"George R.R. Martin , Book:- {i.title}")
session.commit()
session.close()
print("Table created and user added!")

added....
The Books Written by J.K. Rowling are:
Harry Potter and the Philosopher's Stone
Harry Potter and the Chamber of Secrets
The Author Details are:-
George R.R. Martin , Book:- A Game of Thrones
George R.R. Martin , Book:- A Clash of Kings
Table created and user added!


C:\Users\Dell\AppData\Local\Temp\ipykernel_13304\1415007965.py:18: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [111]:
# Task 4: Caching Complex Queries with SQLAlchemy

import time
from cachetools import LRUCache, cached
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
import functools as ft

db_url = 'postgresql+psycopg2://postgres:Admin\
@localhost/postgres'

engine = db.create_engine(db_url)

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

class User(Base):
    __tablename__ = 'Tasks4'
    id = Column(Integer,primary_key=True,autoincrement=True)
    name = Column(String,nullable=False)
    category = Column(String,nullable=False)
    price = Column(Integer,default=0)


Base.metadata.create_all(engine)

products_data = [
   User(name = 'Laptop', category = 'Electronics', price = 1000), 
   User(name = 'Smartphone', category = 'Electronics', price = 700),
   User(name = 'Table', category = 'Furniture', price = 150),
   User(name = 'Chair', category = 'Furniture', price = 85),]

# Insert sample data:
products_data.extend([
   User(name = 'Desk', category = 'Furniture', price = 200),
   User(name = 'Monitor', category = 'Electronics', price = 300),
   User(name = 'Headphones', category = 'Electronics', price = 150),
   User(name = 'Lamp', category = 'Furniture', price = 50),])

session = Session()
# session.add_all(products_data)
session.commit()
session.close()
print("added....")

cache = LRUCache(maxsize = 128)
@cached(cache) 
def querys():
    session = Session()
    result = (session.query(
        User,func.row_number().over(partition_by=User.category,order_by=User.price.desc()).label('rank'))
    ).subquery()
    top_rank = session.query(result).filter(result.c.rank <= 3).all()
    session.close()
    print(top_rank)
    
    return None


session = Session()

#first call
start_time = time.time()
top_rank = querys()
end_time = time.time()
print(f"Time taken for first call: {end_time - start_time} seconds")


#second call
start_time = time.time()
top_rank2 = querys()
end_time = time.time()
print(f"Time taken for second call: {end_time - start_time} seconds")


# Check cache statistics:
# CacheInfo: CacheInfo(hits=1, misses=1, maxsize=128, currsize=1)

# cache_info = querys.cache_info()
# print(f"CacheInfo: {cache_info}")
print(f"Cache size: {len(cache)}")
session.commit()
session.close()
# print("Table created and user added!")


added....
[(1, 'Laptop', 'Electronics', 1000, 1), (9, 'Laptop', 'Electronics', 1000, 2), (2, 'Smartphone', 'Electronics', 700, 3), (5, 'Desk', 'Furniture', 200, 1), (13, 'Desk', 'Furniture', 200, 2), (11, 'Table', 'Furniture', 150, 3)]
Time taken for first call: 0.004991769790649414 seconds
Time taken for second call: 0.0 seconds
Cache size: 1


C:\Users\Dell\AppData\Local\Temp\ipykernel_13304\2838180231.py:19: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [125]:
# Task 5: Implementing Soft Deletes with SQLAlchemy


# from os import abort
import time
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
import functools as ft

db_url = 'postgresql+psycopg2://postgres:Admin\
@localhost/postgres'

engine = db.create_engine(db_url)

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

class User(Base):
    __tablename__ = 'SoftDelete'
    id = Column(Integer,primary_key=True,autoincrement=True)
    name = Column(String,nullable=False)
    age = Column(Integer,default=0)
    deleted = db.Column(db.Boolean(), default=False)

@ft.lru_cache(maxsize=128)
def read_users():
    session = Session()
    active_users = session.query(User).filter(User.deleted == False).all()
    session.close()
    return [(user.name,user.age) for user in active_users]


def delquerys(pid):
    session = Session()
    user = session.query(User).get(pid)
    if user:
        user.deleted = True
        session.commit()
        read_users.cache_clear()
    session.close()
    

Base.metadata.create_all(engine)
users_data = [
   User(name = 'Alice', age = 30), 
   User(name = 'Bob', age = 31), 
   User(name = 'Krut', age = 35),
   User(name = 'Random', age = 40)]

# Insert sample data:
# session.add_all(users_data)
print("added....")



start_time = time.time()
user = read_users()
for name,age in user:
    print(f"{name},{age}")
end_time = time.time()
print(f"Time taken for first call: {end_time - start_time} seconds")


delquerys(2)

start_time = time.time()
user = read_users()
for name,age in user:
    print(f"{name},{age}")
end_time = time.time()
print(f"Time taken for second call: {end_time - start_time} seconds")
    
# Check cache statistics:
cache_info = querys.cache_info()
print(f"CacheInfo: {cache_info}")

session.commit()
print("Table created and user added!")


added....
Alice,30
Bob,31
Krut,35
Random,40
Time taken for first call: 0.008131027221679688 seconds
Alice,30
Krut,35
Random,40
Time taken for second call: 0.0 seconds
CacheInfo: CacheInfo(hits=0, misses=1, maxsize=128, currsize=0)
Table created and user added!


C:\Users\Dell\AppData\Local\Temp\ipykernel_13304\1253817103.py:20: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
C:\Users\Dell\AppData\Local\Temp\ipykernel_13304\1253817103.py:39: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  user = session.query(User).get(pid)
